El sector de Marketing, para tener en cuenta en el envío de campañas, requiere poder realizar una segmentación de clientes, para poder entender sus preferencias y enfocar mejor las campañas.

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set()

In [8]:
from IPython.display import clear_output

Se utiliza el archivo de ventas procesado:

In [10]:
df_ventas = pd.read_csv("../Datasets/Venta_Procesado.csv", sep=",")

In [11]:
df_ventas.head()

,IdVenta,Fecha,Fecha_Entrega,IdCanal,IdCliente,IdSucursal,IdEmpleado,IdProducto,Precio,Cantidad,MinPrecio,MaxPrecio,MinCantidad,MaxCantidad,Calidad_Precio,Calidad_Cantidad,Venta,Fecha_Año,Fecha_Mes,Fecha_Periodo
0,1,2018-03-09,2018-03-17,3,969,13,1674,42817,813.12,2.0,813.12,813.12,0.000,5.000,Sin errores,Sin errores,1626.24,2018,3,201803
1,2,2018-12-28,2018-12-29,2,884,13,1674,42795,543.18,3.0,543.18,543.18,0.875,3.125,Sin errores,Sin errores,1629.54,2018,12,201812
2,3,2016-03-28,2016-03-31,2,1722,13,1674,42837,430.32,1.0,430.32,430.32,0.000,5.000,Sin errores,Sin errores,430.32,2016,3,201603
3,4,2017-10-23,2017-10-24,3,2876,13,1674,42834,818.84,2.0,818.84,818.84,0.500,3.500,Sin errores,Sin errores,1637.68,2017,10,201710
4,5,2017-11-22,2017-11-25,2,678,13,1674,42825,554.18,3.0,554.18,554.18,0.000,5.000,Sin errores,Sin errores,1662.54,2017,11,201711


In [12]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46645 entries, 0 to 46644
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   IdVenta           46645 non-null  int64  
 1   Fecha             46645 non-null  object 
 2   Fecha_Entrega     46645 non-null  object 
 3   IdCanal           46645 non-null  int64  
 4   IdCliente         46645 non-null  int64  
 5   IdSucursal        46645 non-null  int64  
 6   IdEmpleado        46645 non-null  int64  
 7   IdProducto        46645 non-null  int64  
 8   Precio            45721 non-null  float64
 9   Cantidad          45753 non-null  float64
 10  MinPrecio         46645 non-null  float64
 11  MaxPrecio         46645 non-null  float64
 12  MinCantidad       46645 non-null  float64
 13  MaxCantidad       46645 non-null  float64
 14  Calidad_Precio    46645 non-null  object 
 15  Calidad_Cantidad  46645 non-null  object 
 16  Venta             44845 non-null  float6

Se filtran solo los registros sin errores:

In [13]:
df_ventas = df_ventas[(df_ventas['Calidad_Precio'] == 'Sin errores') & (df_ventas['Calidad_Cantidad'] == 'Sin errores')]

In [14]:
df_ventas.shape

(39792, 20)

Se dejan las columnas que se van a utilizar, pero salvando Ids de Cliente y Sucursal todavía:

In [15]:
df_ventas.drop(['IdVenta','Fecha','Fecha_Año','Fecha_Mes','IdProducto','Fecha_Entrega','IdEmpleado','MinPrecio','MaxPrecio','MinCantidad','MaxCantidad','Calidad_Precio','Calidad_Cantidad','Venta','Fecha_Periodo'], axis=1,inplace=True)

In [16]:
df_ventas.head()

,IdCanal,IdCliente,IdSucursal,Precio,Cantidad
0,3,969,13,813.12,2.0
1,2,884,13,543.18,3.0
2,2,1722,13,430.32,1.0
3,3,2876,13,818.84,2.0
4,2,678,13,554.18,3.0


Se lee el maestro de Clientes para obtener la latitud, longitud y edad

In [17]:
df_clientes = pd.read_csv("../Datasets/Clientes.csv", sep=";")

In [18]:
df_clientes.drop(['Provincia','Nombre_y_Apellido','Domicilio','Telefono','Localidad','col10'], axis=1,inplace=True)

Se lee el maestro de Sucursales para obtener latitud y longitud

In [19]:
df_sucursales = pd.read_csv("../Datasets/Sucursales.csv", sep=";")

In [20]:
df_sucursales.drop(['Provincia','Sucursal','Direccion','Localidad'], axis=1,inplace=True)

In [21]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3407 entries, 0 to 3406
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      3407 non-null   int64 
 1   Edad    3407 non-null   int64 
 2   X       3345 non-null   object
 3   Y       3347 non-null   object
dtypes: int64(2), object(2)
memory usage: 106.6+ KB


In [22]:
df_sucursales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        31 non-null     int64 
 1   Latitud   31 non-null     object
 2   Longitud  31 non-null     object
dtypes: int64(1), object(2)
memory usage: 872.0+ bytes


Se puede utilizar la biblioteca Geopy para obtener la distancia entre el cliente y la sucursal:
    !pip install geopy

In [23]:
from geopy.distance import geodesic

Se trabaja sobre los dataframes maestros de clientes y sucursales para adecuar nombres de campos y tipos de datos, tambien para hacer tratamiento sobre valores faltantes

In [24]:
df_sucursales.rename(columns = {'ID': 'IdSucursal', 'Latitud': 'Lat_Sucursal', 'Longitud': 'Lon_Sucursal'}, inplace = True)

In [25]:
df_sucursales['Lat_Sucursal'] = df_sucursales.apply(lambda row: str(row['Lat_Sucursal']).replace(',','.'), axis=1)
df_sucursales['Lon_Sucursal'] = df_sucursales.apply(lambda row: str(row['Lon_Sucursal']).replace(',','.'), axis=1)

In [26]:
df_sucursales = df_sucursales.astype({"Lat_Sucursal": np.float16, "Lon_Sucursal": np.float16})

In [27]:
df_sucursales.describe()

,IdSucursal,Lat_Sucursal,Lon_Sucursal
count,31.000000,31.000000,31.000000
mean,16.000000,-34.250000,-60.375000
std,9.092121,2.347656,3.753906
min,1.000000,-41.125000,-71.312500
25%,8.500000,-34.656250,-60.687500
50%,16.000000,-34.625000,-58.468750
75%,23.500000,-33.734375,-58.406250
max,31.000000,-26.812500,-57.531250


In [28]:
df_clientes.dropna(inplace=True)

In [29]:
df_clientes.shape

(3333, 4)

In [30]:
df_clientes.rename(columns = {'ID': 'IdCliente', 'Y': 'Lat_Cliente', 'X': 'Lon_Cliente'}, inplace = True)

In [31]:
df_clientes['Lat_Cliente'] = df_clientes.apply(lambda row: str(row['Lat_Cliente']).replace(',','.'), axis=1)
df_clientes['Lon_Cliente'] = df_clientes.apply(lambda row: str(row['Lon_Cliente']).replace(',','.'), axis=1)

In [32]:
df_clientes = df_clientes.astype({"Lat_Cliente": np.float16, "Lon_Cliente": np.float16})

In [33]:
df_clientes.describe()

,IdCliente,Edad,Lon_Cliente,Lat_Cliente
count,3333.000000,3333.000000,3333.000000,3333.000000
mean,1710.688269,40.412241,-inf,-inf
std,985.695497,14.577355,14.320312,8.640625
min,1.000000,15.000000,-71.500000,-68.812500
25%,858.000000,28.000000,-60.906250,-34.718750
50%,1721.000000,41.000000,-58.625000,-34.593750
75%,2567.000000,53.000000,-58.437500,-32.937500
max,3407.000000,65.000000,58.843750,34.937500


Como vemos que hay problemas en los datos, es decir, valores de latitud y longitud que no corresponden a Argentina, se procede a hacer correcciones, las cuáles serán de dos tipos:
* Casos donde la latitud y longitud están en positivo
* Casos donde la latitud y longitud están intercambiadas

In [34]:
df_clientes['Lon_Cliente'] = df_clientes['Lon_Cliente'].apply(lambda x: x if x < 0 else x * -1)
df_clientes['Lat_Cliente'] = df_clientes['Lat_Cliente'].apply(lambda x: x if x < 0 else x * -1)

In [35]:
df_clientes.describe()

,IdCliente,Edad,Lon_Cliente,Lat_Cliente
count,3333.000000,3333.000000,3333.000000,3333.000000
mean,1710.688269,40.412241,-60.300119,-34.080202
std,985.695497,14.577355,4.621252,3.609896
min,1.000000,15.000000,-71.500000,-68.812500
25%,858.000000,28.000000,-60.906250,-34.718750
50%,1721.000000,41.000000,-58.656250,-34.593750
75%,2567.000000,53.000000,-58.437500,-32.968750
max,3407.000000,65.000000,-26.734375,-26.593750


In [36]:
df_clientes['Lat_Cliente_Aux'] = df_clientes['Lat_Cliente']

In [37]:
df_clientes['Lat_Cliente'] = df_clientes.apply(lambda row: row['Lat_Cliente'] if row['Lat_Cliente_Aux'] > -55 else row['Lon_Cliente'], axis=1)

In [38]:
df_clientes['Lon_Cliente'] = df_clientes.apply(lambda row: row['Lon_Cliente'] if row['Lat_Cliente_Aux'] > -55 else row['Lat_Cliente_Aux'], axis=1)

In [39]:
df_clientes.describe()

,IdCliente,Edad,Lon_Cliente,Lat_Cliente,Lat_Cliente_Aux
count,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000
mean,1710.688269,40.412241,-60.633935,-33.746386,-34.080202
std,985.695497,14.577355,3.563706,1.988432,3.609896
min,1.000000,15.000000,-71.500000,-40.375000,-68.812500
25%,858.000000,28.000000,-63.968750,-34.718750,-34.718750
50%,1721.000000,41.000000,-58.656250,-34.593750,-34.593750
75%,2567.000000,53.000000,-58.437500,-32.968750,-32.968750
max,3407.000000,65.000000,-57.750000,-26.593750,-26.593750


In [40]:
df_clientes.drop(columns=['Lat_Cliente_Aux'], inplace=True)

Se agrega en el dataframe de ventas los datos de geoposición y edad

In [41]:
df_ventas = pd.merge(df_ventas, df_clientes, on='IdCliente', how='left')

In [42]:
df_ventas = pd.merge(df_ventas, df_sucursales, on='IdSucursal', how='left')

In [43]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39792 entries, 0 to 39791
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   IdCanal       39792 non-null  int64  
 1   IdCliente     39792 non-null  int64  
 2   IdSucursal    39792 non-null  int64  
 3   Precio        39792 non-null  float64
 4   Cantidad      39792 non-null  float64
 5   Edad          38765 non-null  float64
 6   Lon_Cliente   38765 non-null  float64
 7   Lat_Cliente   38765 non-null  float64
 8   Lat_Sucursal  39792 non-null  float16
 9   Lon_Sucursal  39792 non-null  float16
dtypes: float16(2), float64(5), int64(3)
memory usage: 2.9 MB


Los valores nulos pueden ser descartados o bien imputados con otro valor

In [44]:
#df_ventas['Edad'] = df_ventas['Edad'].fillna(0)
#df_ventas['Lon_Cliente'] = df_ventas['Lon_Cliente'].fillna(0)
#df_ventas['Lat_Cliente'] = df_ventas['Lat_Cliente'].fillna(0)

In [45]:
df_ventas.dropna(inplace=True)
df_ventas.shape

(38765, 10)

Se genera el campo donde va a estar la distancia del cliente con la sucursal, que es la geodésica entre los dos puntos de coordenadas

In [46]:
df_ventas['Dist_Cli_Suc'] = df_ventas.apply(lambda row: geodesic((row['Lat_Cliente'], row['Lon_Cliente']),(row['Lat_Sucursal'], row['Lon_Sucursal'])).km, axis=1)

In [47]:
df_ventas['VentaOnline'] = df_ventas.apply(lambda row: 1 if row['IdCanal'] == 2 else 0, axis=1)

In [48]:
df_ventas['Venta'] = df_ventas['Precio'] * df_ventas['Cantidad']

In [49]:
df_ventas.drop(['IdCanal','IdSucursal','Precio','Cantidad','Lon_Sucursal','Lat_Sucursal'], axis=1, inplace=True)

In [50]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38765 entries, 0 to 39791
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   IdCliente     38765 non-null  int64  
 1   Edad          38765 non-null  float64
 2   Lon_Cliente   38765 non-null  float64
 3   Lat_Cliente   38765 non-null  float64
 4   Dist_Cli_Suc  38765 non-null  float64
 5   VentaOnline   38765 non-null  int64  
 6   Venta         38765 non-null  float64
dtypes: float64(5), int64(2)
memory usage: 2.4 MB


In [51]:
df_ventas.head()

,IdCliente,Edad,Lon_Cliente,Lat_Cliente,Dist_Cli_Suc,VentaOnline,Venta
0,969,60.0,-58.28125,-34.81250,35.395419,0,1626.24
1,884,64.0,-58.31250,-34.78125,30.942664,1,1629.54
2,1722,16.0,-58.81250,-34.65625,23.951147,1,430.32
3,2876,19.0,-68.87500,-32.93750,972.361210,0,1637.68
4,678,37.0,-58.65625,-34.71875,16.313885,1,1662.54


In [52]:
df_cliente_dist = df_ventas.groupby(['IdCliente'])['Dist_Cli_Suc'].mean().reset_index()

In [53]:
df_cliente_vta = df_ventas.groupby(['IdCliente'])['Venta'].sum().reset_index()

In [54]:
df_cliente_onl = df_ventas.groupby(['IdCliente'])['VentaOnline'].mean().reset_index()

In [55]:
df_cliente_cant = df_ventas.groupby(['IdCliente'])['Venta'].count().reset_index()

In [56]:
df_cliente_cant.rename(columns = {'Venta': 'CantVentas'}, inplace = True)

In [57]:
df_cliente = df_ventas[['IdCliente','Edad','Lon_Cliente','Lat_Cliente']].drop_duplicates()

In [58]:
df_cliente.rename(columns = {'Lon_Cliente' : 'Longitud', 'Lat_Cliente' : 'Latitud'}, inplace = True)

In [59]:
df_cliente = pd.merge(df_cliente, df_cliente_dist, on='IdCliente')

In [60]:
df_cliente = pd.merge(df_cliente, df_cliente_vta, on='IdCliente')

In [61]:
df_cliente = pd.merge(df_cliente, df_cliente_onl, on='IdCliente')

In [62]:
df_cliente = pd.merge(df_cliente, df_cliente_cant, on='IdCliente')

In [63]:
del(df_ventas)

In [64]:
del(df_sucursales)

In [65]:
del(df_clientes)

In [66]:
del(df_cliente_dist)

In [67]:
del(df_cliente_vta)

In [68]:
del(df_cliente_onl)

In [69]:
del(df_cliente_cant)

In [70]:
df_cliente.describe()

,IdCliente,Edad,Longitud,Latitud,Dist_Cli_Suc,Venta,VentaOnline,CantVentas
count,2896.000000,2896.000000,2896.000000,2896.000000,2896.000000,2896.000000,2896.000000,2896.000000
mean,1624.626727,40.388122,-60.417764,-33.987127,282.057468,33909.248042,0.455339,13.385704
std,960.037330,14.599569,3.586638,1.781704,303.743916,28362.139705,0.212828,7.935451
min,1.000000,15.000000,-71.500000,-40.375000,0.000000,113.000000,0.000000,1.000000
25%,815.750000,28.000000,-60.625000,-34.718750,26.030742,12725.170000,0.333333,7.000000
50%,1592.500000,41.000000,-58.625000,-34.593750,190.016525,27272.570000,0.450000,13.000000
75%,2415.250000,53.000000,-58.437500,-34.031250,383.237593,47538.935000,0.578947,19.000000
max,3407.000000,65.000000,-57.750000,-26.593750,1381.902436,206206.360000,1.000000,46.000000


Se generan los datos con los que se va a entrenar K-Means para generar los Clústers

In [71]:
X = None

In [72]:
from sklearn.cluster import KMeans

In [73]:
from sklearn.metrics import silhouette_samples, silhouette_score

Se Evalua por Distancia media al Centroide y Silhouette

Se grafica la distancia media en función del núemro de clústers.

Se grafica Silhouette media en función del núemro de clústers.